# 校车行程时间数据挖掘

- openstreetmap下载同济大学嘉定校区、同济大学沪西校区，同济大学四平校区路网数据
- arcgis建立块和缓冲区
- 利用arcpy，判断是否进入到小区内部
- 计算一次出行的行程时间
- ...数据挖掘，统计信息分析

In [ ]:
%pylab
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import os
sns.set()

## 1. 数据初始处理
- 文件中的中文词汇影响读取 改名
- 检查原始数据文件列名 统一格式
- 更改日期时间格式 便于筛选

In [ ]:
# remove Chinese words
path = 'C:\\Users\\D\\Desktop\\banche\\'
for name in os.listdir(path):
    os.rename(path+name, path + name[1:])
os.listdir(path)

In [ ]:
for i in os.listdir(path):
    df = pd.read_csv(path + i, encoding='gbk')
    print(df.columns)

**<big>原始数据 'D05057 2.csv' 文件 year 列名后面加了个空格！！！</big>**

In [ ]:
# 日期时间格式处理
# 时间统一改成24小时制
# 日期改成yyyymmdd


def df_add_time(df):
    df = df[['UID', 'rectime', 'year', 'date1', 'time', 'time1', 'speed','longitude', 'latitude', 'location', 'LID']]
    # 处理日期
    def f(x):
        sp = x.split('/')
        y = sp[-1]
        m = sp[0]
        d = sp[1]
        if len(m)==1:
            m = '0'+ m
        if len(d)==1:
            d = '0' + d
        return int(y+m+d)
    df['rdate'] = df.rectime.apply(f)
    # 处理时间
    def f(x):
        sp = re.split(':| ',x)
        h = sp[0]
        m = sp[1]
        s = sp[2]
        if sp[3] == 'PM':
            if not int(h) == 12:
                h = str(int(h) + 12)
        else :
            if int(h) == 12:
                h = str(int(h) - 12)
        return int(h + m+ s)
    df['rtime'] = df.time.apply(f)
    df = df.drop(labels=['time1','date1','UID'],axis=1)
    return df

In [ ]:
for i in os.listdir(path):
    df = pd.read_csv(path + i, encoding='gbk')
    df = df_add_time(df)
    df.to_csv('C:\\Users\\D\\Desktop\\banche1\\' + i, encoding='gbk', index=False)

## 2. 数据理解

In [ ]:
print(df.rdate.unique())
print(len(df.rdate.unique()))

日期117个

In [ ]:
df.LID.unique()

1 - 沪西校区
2 - 嘉定校区
3 - 四平校区
4 - 曹杨新村
<b>位置是否反编码得到，如果仅仅临近校区内是否就有比较大的误差？

In [ ]:
plot(df[df.rdate == 20170919].rtime)

- 基本每隔十秒回传一个数据
- 如果车辆停止，那么会每隔一个小时回传一个数据

## 3. 统计行程时间信息

统计思路：
1. 按日期做循环，统计一天内的出行次数
2. 如果连续有60个数据的lid为0，也就是10分钟左右不在学校，那么作为一次出行
    - 实际处理中可能由于途径沪西校区或者曹阳新村，会是一次出行打断，处理方法是判断两次较大时长出行的首尾数据是否临近，取阀值为30个位置，也即5分钟左右，临近的话合并为一次出行
3. 记录这次出行的起点位置index，终点位置index
4. 起终点index分别向上下加减一 ，直到找到一条位置在同济大学校区内且速度为0的数据
    - 如果在一天的数据中找不到头和尾，那么就把头和尾数据记为空值
5. 取4中找到数据的时间位置分别作为开始时间，开始位置，结束时间，结束位置
    - 为了检查异常数据，顺便记录下来在找到的数据在原始数据中的位置


目标表格列名   
- 基本数据
    - 车牌    id
    - 日期    date
    - 开始时间   start_time
    - 到达时间   end_time
    - 出发地点   start_lid
    - 到达地点   end_lid
    - 途经曹阳   caoyang
- 拓展数据
    - 线路
    - 总行程时间
    - 如何找到一次行程的全体轨迹数据？
        - id
        - date
        - index  

In [ ]:
new_path = 'C:\\Users\\D\\Desktop\\banche1\\'
for name in os.listdir(new_path):
    df = pd.read_csv(new_path + name, encoding='gbk')
    print(df[:1])

In [ ]:
# find_path v1版本
def find_path(df, name):
    Dic = {'date':[],'start_time':[],'start_lid':[],'end_time':[],'end_lid':[],'caoyang':[]}
    dates = df.rdate.unique()
    for date in dates:
        m = df[df.rdate == date]
        m = m.reset_index()
        lid = array(m.LID.tolist())
        trip = np.where(lid==0)[0]


        # 得到一天的出行次数
        from itertools import groupby
        dic = {}
        last_index = -1000
        fun = lambda x: x[1] - x[0]
        for key, group in groupby(enumerate(trip), fun):
            lst = [v for i,v in group]
            if len(lst)>30: 
                if lst[0] - last_index < 30:
                    dic[last_key].extend(lst)
                else:
                    dic[key] = lst
                    last_key = key
                    last_index = lst[-1]


        # 画图为了后续检验比较纠错有几条明显折线就有几次出行
        figure(figsize=(6,6))
        axis('off')
        plot(trip)
        plt.savefig('F:\\xiaochefig\\{}-{}-{}.jpg'.format(name, date, len(dic)))
        
        
        # 找到对应起终点路径
        # 出发到达时间 出发到达位置 是否经过曹阳
        for key in dic.keys():
            start_index = dic[key][0]
            end_index = dic[key][-1]
            while 1:
                if m.loc[start_index].speed == 0 and m.loc[start_index].LID in [1,2,3]:
                    start_time = m.loc[start_index].rtime
                    start_lid = m.loc[start_index].LID
                    break
                else:
                    start_index = start_index - 1
                    if start_index <= 0:
                        start_time = NaN
                        start_lid = NaN
                        print("_________something wrong in start!______")
                        print(m.loc[dic[key][0]])
                        print("________________________________________")
                        break
            while 1:
                if m.loc[end_index].speed == 0 and m.loc[end_index].LID in [1,2,3]:
                    end_time = m.loc[end_index].rtime
                    end_lid = m.loc[end_index].LID
                    break
                else:
                    end_index = end_index + 1
                    if end_index >= len(m):
                        end_time = NaN
                        end_lid = NaN
                        print("_________something wrong in end!______")
                        print(m.loc[dic[key][0]])
                        print("______________________________________")
                        break
            caoyang = 1 if 4 in m.loc[start_index:end_index].LID.unique() else 0
            Dic['date'].append(date)
            Dic['start_time'].append(start_time)
            Dic['start_lid'].append(start_lid)
            Dic['end_time'].append(end_time)
            Dic['end_lid'].append(end_lid)
            Dic['caoyang'].append(caoyang)
    return Dic

In [ ]:
# find_path v2版本
# 增添了原始数据位置信息
# 增添了异常信息的记录
def find_path(df, name):
    error_log = ''
    Dic = {'start_index': [], 'end_index': [], 'date': [], 'start_time': [], 'start_lid': [], 'end_time': [],
           'end_lid': [], 'caoyang': []}
    dates = df.rdate.unique()
    for date in dates:
        m = df[df.rdate == date]
        m = m.reset_index()
        lid = array(m.LID.tolist())
        trip = np.where(lid == 0)[0]

        # 得到一天的出行次数
        from itertools import groupby
        dic = {}
        last_index = -1000
        fun = lambda x: x[1] - x[0]
        for key, group in groupby(enumerate(trip), fun):
            lst = [v for i, v in group]
            if len(lst) > 60:
                if lst[0] - last_index < 30:
                    dic[last_key].extend(lst)
                else:
                    dic[key] = lst
                    last_key = key
                    last_index = lst[-1]

        # 画图为了后续检验比较纠错有几条明显折线就有几次出行
        figure(figsize=(6, 6))
        axis('off')
        plot(trip)
        savefig('F:\\xiaochefig\\{}-{}-{}.jpg'.format(name[0:-4], date, len(dic)))

        # 找到对应起终点路径
        # 出发到达时间 出发到达位置 是否经过曹阳
        for key in dic.keys():
            start_index = dic[key][0]
            end_index = dic[key][-1]
            while 1:
                if m.loc[start_index].speed == 0 and m.loc[start_index].LID in [1, 2, 3]:
                    start_time = m.loc[start_index].rtime
                    start_lid = m.loc[start_index].LID
                    break
                else:
                    start_index = start_index - 1
                    if start_index <= 0:
                        start_time = NaN
                        start_lid = NaN
                        print("_________something wrong in start!______")
                        print(m.loc[dic[key][0]])
                        print("________________________________________")
                        error_log = error_log + '_________something wrong in start!______\n'
                        error_log = error_log + 'chepai: {}\n'.format(name[0:-4])
                        error_log = error_log + str(m.loc[dic[key][0]]) + '\n'
                        error_log = error_log + '________________________________________\n'
                        break
            while 1:
                if m.loc[end_index].speed == 0 and m.loc[end_index].LID in [1, 2, 3]:
                    end_time = m.loc[end_index].rtime
                    end_lid = m.loc[end_index].LID
                    break
                else:
                    end_index = end_index + 1
                    if end_index >= len(m):
                        end_time = NaN
                        end_lid = NaN
                        print("_________something wrong in end!______")
                        print(m.loc[dic[key][0]])
                        print("______________________________________")
                        error_log = error_log + '_________something wrong in end!______\n'
                        error_log = error_log + 'chepai: {}\n'.format(name[0:-4])
                        error_log = error_log + str(m.loc[dic[key][-1]]) + '\n'
                        error_log = error_log + '________________________________________\n'
                        break
            caoyang = 1 if 4 in m.loc[start_index:end_index].LID.unique() else 0
            Dic['start_index'].append(m.loc[dic[key][0]]['index'])
            Dic['end_index'].append(m.loc[dic[key][-1]]['index'])
            Dic['date'].append(date)
            Dic['start_time'].append(start_time)
            Dic['start_lid'].append(start_lid)
            Dic['end_time'].append(end_time)
            Dic['end_lid'].append(end_lid)
            Dic['caoyang'].append(caoyang)
    return Dic, error_log

In [ ]:
# test code
Dic = find_path(df,'test')

In [ ]:
# 统计行程时间信息和错误日志
with open('C:\\Users\\D\\Desktop\\log.txt', 'w') as f:
    new_path = 'C:\\Users\\D\\Desktop\\banche1\\'
    for name in os.listdir(new_path):
        df = pd.read_csv(new_path + name, encoding='gbk')
        Dic, error_log = find_path(df, name)
        info = pd.DataFrame(Dic)
        info['id'] = name[0:-4]
        info = info[
            ['id', 'date', 'start_index', 'end_index', 'start_time', 'start_lid', 'end_time', 'end_lid', 'caoyang']]
        info.to_csv('F:\\info\\{}.csv'.format(name), index=False)
        f.write(error_log)

## 4. 异常数据寻因

结论  
一些不符合逻辑的数据
- 四平校区到嘉定校区总是会经过沪西校区不停，带来一些行程的断开
- 出去加了个油又回到之前的校区了
- 沪DG6217 2 的车行日期较少，带来结果偏少
- 中途才开GPS，出发时刻不确定，无法与时刻表对应D03923 [1086:1216]

In [ ]:
m = df[df.rdate==20171111]
m = m.reset_index()
m

In [ ]:
for key in dic.keys():
    start_index = dic[key][0]
    end_index = dic[key][-1]
    print(start_index,end_index)

In [ ]:
m.loc[0:657].to_csv('C:\\Users\\D\\Desktop\\wen.csv',encoding='gbk')

In [ ]:
with open('C:\\Users\\D\\Desktop\\change_loc.txt','w') as f:
    for i in last:
        f.write(str(i))
        f.write('\n')

In [ ]:
m[150:151]

In [ ]:
s = pd.read_csv('F:\\info\\D03923 2.csv.csv')

In [ ]:
new_path = 'C:\\Users\\D\\Desktop\\banche1\\'
df = pd.read_csv(new_path + 'D03923 2.csv',encoding = 'gbk')

In [ ]:
m1 = s[(s.start_lid==1)&(s.end_lid==2)]
len(m1[(abs(m1.start_time-71000)<500)])

In [ ]:
m1

In [ ]:
df.loc[300816:300985]

## 5. 行程时间按时刻表分类

In [ ]:
s = os.listdir('C:\\Users\\D\\Desktop\\代码处理后数据\\')

In [ ]:
code_route = [int(i.split('.')[0]) for i in s]

In [ ]:
route = pd.read_excel('C:\\Users\\D\\Desktop\\route.xlsx')

In [ ]:
real_route = route.no.tolist()

In [ ]:
[i for i in code_route if i not in real_route]

In [ ]:
[i for i in real_route if i not in code_route]

In [ ]:
import datetime
for i in s:
    code = pd.read_csv(u'C:\\Users\\D\\Desktop\\codedata\\{}'.format(i))
    if len(code.columns)>9:
        continue
    code.columns = ['Unnamed', 'id', 'ID', 'ST', 'AT', 'DT', 'LO', 'RO', 'TT']
    code = code[['id', 'ID', 'ST', 'AT', 'DT', 'LO', 'RO', 'TT']]

    def fun(x):
        if x > 40:
            time = datetime.datetime(2017, 1, 1) + datetime.timedelta(x-1)
        else:
            time = datetime.datetime(2018, 1, 1) + datetime.timedelta(x-1)
        return time
    code['dt'] = code.DT.apply(fun)
    
    def fun(x):
        s = x%3600%60
        m = (x%3600 - s)/60
        h = (x - x%3600)/3600
        return "{}:{}:{}".format(int(h),int(m),int(s))
    code['st'] = code.ST.apply(fun)
    code['at'] = code.AT.apply(fun)
    code['tt'] = code.TT.apply(fun)
    code.to_csv(u'C:\\Users\\D\\Desktop\\codedata\\{}'.format(i),index=False)

In [ ]:
def fun1(x):
    if isnan(x):
        return NaN
    else:
        x = int(x)
        h = x//10000
        m = (x - 10000*h)//100
        s = x%100
    return h*3600 + m*60 + s
def fun2(x):
    if isnan(x):
        return NaN
    else:
        s = x%3600%60
        m = (x%3600 - s)/60
        h = (x - x%3600)/3600
        return "{}:{}:{}".format(int(h),int(m),int(s))
for i in os.listdir("C:\\Users\\D\\Desktop\\info\\"):
    path = "C:\\Users\\D\\Desktop\\info\\" + i
    info = pd.read_csv(path)
    info['total_time'] = info.end_time.apply(fun1) - info.start_time.apply(fun1)
    info['total_time_format'] = info.total_time.apply(fun2) 
    info.to_csv(path,index=False)

In [ ]:
result = pd.DataFrame()
for i in os.listdir("C:\\Users\\D\\Desktop\\info\\"):
    path = "C:\\Users\\D\\Desktop\\info\\" + i
    info = pd.read_csv(path)
    find = info[(abs(info.start_time - 124500))<500]
    find = find[(find.start_lid == 1)&(find.end_lid == 2)]
    result = pd.concat([result,find])
result

In [ ]:
# 一般route
def fun1(x):
    if isnan(x):
        return NaN
    else:
        x = int(x)
        h = x//10000
        m = (x - 10000*h)//100
        s = x%100
    return h*3600 + m*60 + s
for i in range(0,len(route)):
    result = pd.DataFrame()
    no = route.loc[i]['code_no']
    t = route.loc[i]['time']
    slid = route.loc[i]['slid']
    elid = route.loc[i]['elid']
    if slid == 0:
        continue
    for name in os.listdir("C:\\Users\\D\\Desktop\\info\\"):
        path = "C:\\Users\\D\\Desktop\\info\\" + name
        info = pd.read_csv(path)
        find = info[abs(info.start_time.apply(fun1) - route.loc[i]['time'].hour*3600 - route.loc[i]['time'].minute*60)<=300]
        find = find[(find.start_lid == slid)&(find.end_lid == elid)]
        result = pd.concat([result,find])
    result.to_csv("C:\\Users\\D\\Desktop\\route\\{}.csv".format(no),index=False)

In [ ]:
# 特殊route
result = pd.DataFrame()
no = 2312
t = datetime.time(8,0)
slid = 2
elid = 3
for name in os.listdir("C:\\Users\\D\\Desktop\\info\\"):
    path = "C:\\Users\\D\\Desktop\\info\\" + name
    info = pd.read_csv(path)
    find = info[abs(info.start_time.apply(fun1) - t.hour*3600 - t.minute*60)<=300]
    find = find[(find.start_lid == slid)&(find.end_lid == elid)]
    find = find[find.caoyang == 1]
    result = pd.concat([result,find])
result.to_csv("C:\\Users\\D\\Desktop\\route\\{}.csv".format(no),index=False)

In [ ]:
# 特殊route
result = pd.DataFrame()
no = 323
t1 = datetime.time(8,0)
t2 = datetime.time(8,45)
slid = 3
elid = 2
for name in os.listdir("C:\\Users\\D\\Desktop\\info\\"):
    path = "C:\\Users\\D\\Desktop\\info\\" + name
    info = pd.read_csv(path)
    find = info[info.start_time.apply(fun1) - t1.hour*3600 - t1.minute*60 + 300>=0]
    find = find[find.start_time.apply(fun1) - t2.hour*3600 - t2.minute*60 - 300<=0]
    find = find[(find.start_lid == slid)&(find.end_lid == elid)]
    find = find[find.caoyang == 1]
    result = pd.concat([result,find])
result.to_csv("C:\\Users\\D\\Desktop\\route\\{}.csv".format(no),index=False)